# Load Open Street Maps Data

This notebook is aimed to demonstrate how we reach spartial data on power transmission lines. Our main data source are the Open Street Maps datasets. 

In [1]:
import sys
sys.path.append('../')  # to import helpers

from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

This is the repository path:  ./
Had to go 1 folder(s) up.


## Import nessesary packages

Load Python packages and set visibility options:

In [2]:
import logging
import sys
import pandas as pd
import requests
import urllib3
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 70)

logger = logging.getLogger(__name__)

Load local packages written to load OSM data:

In [3]:
from scripts.config_osm_data import continent_regions
from scripts.config_osm_data import continents
from scripts.config_osm_data import iso_to_geofk_dict
from scripts.config_osm_data import world_iso
from scripts.config_osm_data import world_geofk

## Management of geographical data

OSM data are being organized by continents, macroregions and countries. Input data on country codes should correspond to ISO standard and be transformed into a valid OSM data request.

The `world_geofk` and `world_iso` Python two-levels dictionaries are used to keep data on such organization according to OSM and ISO conventions, respectivelly. Define a couple of supplementary functions to work with these data structures. The first one `list_countries()` transforms an input dictionary into a list while the second `getContinentCountryIso()` retrives the continent and country names by the country code.

In [4]:
def list_countries(w_dc):
    countries_list = []

    for continent in w_dc:
        country = w_dc[continent]
        countries_list.append(list(country.keys()))
        
    return countries_list 

def getContinentCountryIso(code):
    for continent in world_iso:
        country = world_iso[continent].get(code, 0)
        if country:
            return continent, country
    return continent, country

list_word_iso_countries = list_countries(world_iso)
list_word_geofk_countries = list_countries(world_geofk)

iso_set = set(sum(list_word_iso_countries, []))
geofk_set = set(sum(list_word_geofk_countries, []))

iso_not_in_geofk = iso_set - geofk_set
geofk_not_in_iso = geofk_set - iso_set

print("GeoFabrik naming differs from ISO for the following countries:")
for cnt in list(iso_not_in_geofk):
    print(getContinentCountryIso(cnt))

lost_codes = set(iso_to_geofk_dict.keys()) - set(iso_not_in_geofk)
    
print("Any ISO codes not resolved by GeoFbk and a transform dictionary?")
if len(lost_codes) > 0:
    print(lost_codes)
    for cnt in list(lost_codes):
        print(getContinentCountryIso(cnt))
else:
    print("...everithing seems to be allright")

GeoFabrik naming differs from ISO for the following countries:
('asia', 'macao')
('asia', 'saudi-arabia')
('asia', 'united-arab-emirates')
('latin_america', 'french-guyane')
('asia', 'qatar')
('asia', 'palestine')
('latin_america', 'guyane')
('asia', 'malaysia')
('asia', 'oman')
('asia', 'kuwait')
('asia', 'hong kong')
('africa', 'gambia')
('asia', 'bahrain')
('africa', 'western-sahara')
('africa', 'senegal')
('europe', 'san-marino')
('asia', 'israel')
('asia', 'singapore')
('asia', 'brunei')
Any ISO codes not resolved by GeoFbk and a transform dictionary?
...everithing seems to be allright


Look on the macro regions

In [5]:
macro_regions_list = list(dict(**continent_regions).values())
# flatten list
macro_reg_set = set(sum(macro_regions_list, []))


print("ISO codes not included into the macro regions dictionary:")
print(macro_reg_set - iso_set)
print(iso_set - macro_reg_set)

print(len(macro_reg_set))
print(len(iso_set))

print(list(iso_set - macro_reg_set))

for cnt in list(iso_set - macro_reg_set):
    print(getContinentCountryIso(cnt))

ISO codes not included into the macro regions dictionary:
set()
{'PG', 'AU', 'GF', 'EH', 'MT', 'GW', 'PY', 'NZ', 'SN', 'NC', 'SO', 'GY'}
158
170
['PG', 'AU', 'GF', 'EH', 'MT', 'GW', 'PY', 'NZ', 'SN', 'NC', 'SO', 'GY']
('australia', 'papua-new-guinea')
('australia', 'australia')
('latin_america', 'french-guyane')
('africa', 'western-sahara')
('europe', 'malta')
('africa', 'guinea-bissau')
('latin_america', 'paraguay')
('australia', 'new-zealand')
('africa', 'senegal')
('australia', 'new-caledonia')
('africa', 'somalia')
('latin_america', 'guyane')


In [6]:
print(set(continents))

print(set(continents) - iso_set)
print(set(continents) - geofk_set)

{'SA', 'AS', 'LA', 'OC', 'CA', 'EU', 'AF'}
{'AS', 'OC', 'EU'}
{'SA', 'AS', 'EU', 'OC'}


In [7]:
def getContinentCountry(code):
    for continent in world_geofk:
        country = world_geofk[continent].get(code, 0)
        if country:
            return continent, country
    return continent, country

def build_url(country_code, update, verify):
    continent, country_name = getContinentCountry(country_code)
    geofabrik_filename = f"{country_name}-latest.osm.pbf"
    geofabrik_url = f"https://download.geofabrik.de/{continent}/{geofabrik_filename}"
    return geofabrik_url


In [8]:
problem_urls = []
problem_codes = []
problem_domain = []

# flatten list
world_geofk_codes = sum(list_word_iso_countries, [])

for cnt in world_geofk_codes[0:9]:    
    print(getContinentCountry(cnt))
    url = build_url(country_code=cnt, update=False, verify=False)
    print(url)
    time.sleep(1)
    
    with requests.get(url, stream=True, verify=True) as r:
        request = requests.head(url)
        if r.status_code == 200:
            print("URL '" + url + "' is working")
        else:
            problem_urls.append(url)
            problem_codes.append(cnt)
            problem_domain.append(getContinentCountry(cnt))
            
            if r.status_code == 429:
                print("Error code:" + str(r.status_code) + " Retry after" + r.headers["Retry-After"])
            else:
                print("There some troubles with " + url + " Error code:" + str(r.status_code))

if len(problem_urls) > 0:              
    print("There were troubles in reaching following urls:") 
    print(problem_urls) 
    print("Country codes to be checked:")
    print(problem_codes) 
    print(problem_domain)

('africa', 'algeria')
https://download.geofabrik.de/africa/algeria-latest.osm.pbf
URL 'https://download.geofabrik.de/africa/algeria-latest.osm.pbf' is working
('africa', 'angola')
https://download.geofabrik.de/africa/angola-latest.osm.pbf
URL 'https://download.geofabrik.de/africa/angola-latest.osm.pbf' is working
('africa', 'benin')
https://download.geofabrik.de/africa/benin-latest.osm.pbf
URL 'https://download.geofabrik.de/africa/benin-latest.osm.pbf' is working
('africa', 'botswana')
https://download.geofabrik.de/africa/botswana-latest.osm.pbf
URL 'https://download.geofabrik.de/africa/botswana-latest.osm.pbf' is working
('africa', 'burkina-faso')
https://download.geofabrik.de/africa/burkina-faso-latest.osm.pbf
URL 'https://download.geofabrik.de/africa/burkina-faso-latest.osm.pbf' is working
('africa', 'burundi')
https://download.geofabrik.de/africa/burundi-latest.osm.pbf
URL 'https://download.geofabrik.de/africa/burundi-latest.osm.pbf' is working
('africa', 'cameroon')
https://downlo


# Acknowledgments

The project relies on the (OpenStreetMap)[https://www.geofabrik.de/] data provided via Geofabrik service. Many thanks to all the service contributors.